#Information
Group 9 : Resume parser

Content : Classification of resumes using TF-IDF

In [2]:
# import packages

import pandas as pd
import numpy as np
import seaborn as sns
import re
import numpy as np
import math
import pickle
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# Connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Move the directory to the csv files location
%cd /content/drive/MyDrive/NLP_Final_changes/

/content/drive/MyDrive/NLP_Final_changes


In [8]:
ls

Bert_classification.ipynb                 sample_java_job_description.docx
Bert_Similarity.ipynb                     sample_java_resumes.csv
Category_plot.png                         sample_testing_job_description.docx
cleaned_resume_final.csv                  sample_testing_resumes.csv
Preprocessing.ipynb                       TF_IDF_Classification.ipynb
resume_classification_model/              TF_IDF_Similarity.ipynb
ResumeDataSet.csv                         Word2Vec.ipynb
sample_data_science_job_description.docx  Word_cloud.png
sample_data_science_resumes.csv


In [10]:
# Load required CSV files
df=pd.read_csv("cleaned_resume_final.csv")

In [11]:
# Labeling categories with an unique number
df['Category_no'] = df.groupby('Category').ngroup()

In [12]:
df.head()

,Unnamed: 0,Category,Resume,cleaned_resume,Category_no
0,0,Data Science,Skills * Programming Languages: Python (pandas...,skill programming language python panda numpy ...,6
1,1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education detail may may data scientist data s...,6
2,2,Data Science,"Areas of Interest Deep Learning, Control Syste...",area interest deep learning control system des...,6
3,3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skill r python sap hana tableau sap hana sql s...,6
4,4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education detail mca ymcaust faridabad haryana...,6


In [13]:
# Remove unnamed column
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [16]:
df = df.drop_duplicates(subset=['cleaned_resume'])

In [17]:
df.shape

(166, 4)

In [18]:
# split train and test data. Here we are using 80% train data and 20% test data. You can change based on your requirement
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.cleaned_resume,
    df.Category_no,
    test_size = 0.2,
    random_state=1,
    stratify=df.Category_no
)

In [19]:
# Sizes of train and test data
print("Shape of X_train : ", X_train.shape)
print("Shape of X_test : ", X_test.shape)

Shape of X_train :  (132,)
Shape of X_test :  (34,)


In [20]:
# import necessary packages
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Using TfidfVectorizer() to vectorize the string data into vectors
# Passing the output vectors to KNeighborsClassifier() to generate classification
# KNN classifier
clf_knn = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('KNN', KNeighborsClassifier())
])

clf_knn.fit(X_train, y_train)
# Pickle is used to save the data and can be diretly used again without running the code again
# Uncomment the below line to pickle the data after passing the required location

#pickle.dump(clf_knn, open('/content/drive/MyDrive/NLP/pkl/tfidf_knn.pkl', 'wb'))
y_pred_knn = clf_knn.predict(X_test)

# Printing classification report
print(classification_report(y_test, y_pred_knn))

              precision    recall  f1-score   support

           0       0.67      1.00      0.80         2
           1       0.00      0.00      0.00         1
           2       0.25      1.00      0.40         1
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         1
           6       0.33      0.50      0.40         2
           7       1.00      1.00      1.00         2
           8       0.00      0.00      0.00         2
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         1
          12       1.00      1.00      1.00         2
          13       1.00      1.00      1.00         1
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         3
          16       1.00      1.00      1.00         1
          17       1.00    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
# Note: you can use your own resume to test it. But it needs to be preprocessed before passing for prediction
# Testing the model
test = []
test.append(df['cleaned_resume'][0])

y_pred_test = clf_knn.predict(test)
print('Category of sample resume is : ', df['Category'][0])
print('Category of predicted resume is : ', df.loc[df['Category_no'] == y_pred_test[0]]['Category'].unique()[0])

Category of sample resume is :  Data Science
Category of predicted resume is :  Data Science


In [25]:
# Gaussian naive bayes classifier
from sklearn.naive_bayes import GaussianNB
from mlxtend.preprocessing import DenseTransformer

clf_gnb = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('to_dense', DenseTransformer()),
    ('GNB', GaussianNB())
])

clf_gnb.fit(X_train, y_train)

# Uncomment the below line to pickle the data after passing the required location

#pickle.dump(clf_gnb, open('/content/drive/MyDrive/NLP/pkl/tfidf_gnb.pkl', 'wb'))

y_pred_gnb = clf_gnb.predict(X_test)

print(classification_report(y_test, y_pred_gnb))

              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.00      0.00      0.00         1
           2       0.20      1.00      0.33         1
           3       1.00      1.00      1.00         1
           4       0.00      0.00      0.00         1
           5       0.25      1.00      0.40         1
           6       0.00      0.00      0.00         2
           7       1.00      0.50      0.67         2
           8       0.33      0.50      0.40         2
           9       1.00      1.00      1.00         2
          10       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         2
          13       0.33      1.00      0.50         1
          14       1.00      1.00      1.00         1
          15       1.00      0.67      0.80         3
          16       0.00      0.00      0.00         1
          17       1.00    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
# Note: you can use your own resume to test it. But it needs to be preprocessed before passing for prediction
# Testing the model
test = []
test.append(df['Resume'][0])

y_pred_test = clf_gnb.predict(test)
print('Category of sample resume is : ', df['Category'][0])
print('Category of predicted resume is : ', df.loc[df['Category_no'] == y_pred_test[0]]['Category'].unique()[0])

Category of sample resume is :  Data Science
Category of predicted resume is :  Data Science


In [28]:
# Random forrest classifier
from sklearn.ensemble import RandomForestClassifier
clf_rf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('Random Forest', RandomForestClassifier())
])

clf_rf.fit(X_train, y_train)

# Uncomment the below line to pickle the data after passing the required location

#pickle.dump(clf_rf, open('/content/drive/MyDrive/NLP/pkl/tfidf_rf.pkl', 'wb'))

y_pred_rf = clf_rf.predict(X_test)

print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       0.50      1.00      0.67         1
           2       0.00      0.00      0.00         1
           3       1.00      1.00      1.00         1
           4       0.25      1.00      0.40         1
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         2
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         1
          12       0.40      1.00      0.57         2
          13       1.00      1.00      1.00         1
          14       1.00      1.00      1.00         1
          15       0.75      1.00      0.86         3
          16       0.00      0.00      0.00         1
          17       1.00    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
# Testing the model
test = []
test.append(df['Resume'][0])

y_pred_test = clf_rf.predict(test)
print('Category of sample resume is : ', df['Category'][0])
print('Category of predicted resume is : ', df.loc[df['Category_no'] == y_pred_test[0]]['Category'].unique()[0])

Category of sample resume is :  Data Science
Category of predicted resume is :  Data Science
